# Método Triaxial — Notebook 04  
## Casos legales hipotéticos con matriz 3×N (F–C–P)

Este notebook muestra cómo aplicar el **Método Triaxial de Discernimiento**  
a **casos legales hipotéticos**, usando la matriz completa de 15 criterios:

- 5 criterios de **Fundamento (F)**  
- 5 criterios de **Contexto (C)**  
- 5 criterios de **Principio (P)**  

⚠️ Todos los casos son **hipotéticos**, construidos a partir de patrones frecuentes
en noticias, procesos legales y conflictos sociales, pero **no corresponden** a personas
ni expedientes reales.

Objetivos de este notebook:

1. Simular cómo se evaluaría un caso legal usando los 15 criterios del método.  
2. Comparar distintos tipos de casos (ej. denuncia sólida vs. denuncia mediática).  
3. Dejar una base didáctica para cursos, talleres o análisis académico.


In [1]:
from dataclasses import dataclass
from typing import List, Dict
import pandas as pd


@dataclass
class Criterion:
    axis: str          # 'F', 'C' o 'P'
    code: str          # F1, F2, ..., P5
    name: str          # Nombre corto
    question: str      # Pregunta / criterio operativo


# 15 criterios del cuadro de evaluación
CRITERIA: List[Criterion] = [
    # FUNDAMENTO (F)
    Criterion("F", "F1", "Fuente confiable", 
              "La fuente es confiable / verificable"),
    Criterion("F", "F2", "Consistencia con hechos", 
              "La afirmación es consistente con hechos conocidos"),
    Criterion("F", "F3", "Funcionamiento real del mundo", 
              "Coincide con el funcionamiento real del mundo"),
    Criterion("F", "F4", "Sin suposiciones forzadas", 
              "No requiere suposiciones forzadas para sostenerse"),
    Criterion("F", "F5", "Sin contradicción fuerte", 
              "No contradice evidencia sólida"),

    # CONTEXTO (C)
    Criterion("C", "C1", "Contexto claro", 
              "Existe contexto claro y transparente"),
    Criterion("C", "C2", "Sin presión emocional/manipulación", 
              "No proviene de emoción, presión o manipulación"),
    Criterion("C", "C3", "Sin estigmatizar grupos", 
              "No generaliza ni estigmatiza grupos completos"),
    Criterion("C", "C4", "Sin agendas interesadas", 
              "Está aislada de agendas políticas o mediáticas interesadas"),
    Criterion("C", "C5", "Marco realista", 
              "Los datos o hechos están situados en un marco realista"),

    # PRINCIPIO (P)
    Criterion("P", "P1", "Menor entropía", 
              "Produce menor entropía (menos caos/confusión)"),
    Criterion("P", "P2", "Coherencia interna y externa", 
              "Fomenta coherencia interna y externa"),
    Criterion("P", "P3", "Respeto a la dignidad", 
              "Respeta la dignidad humana e institucional"),
    Criterion("P", "P4", "Sin odio ni división", 
              "No alimenta odio, división o violencia"),
    Criterion("P", "P5", "Acción responsable", 
              "Abre caminos de acción responsable"),
]


def criteria_template() -> pd.DataFrame:
    """Devuelve una tabla con los criterios y columna 'valor' en blanco."""
    df = pd.DataFrame([c.__dict__ for c in CRITERIA])
    df["valor"] = None
    return df


def decide_action(D: float) -> str:
    """Misma regla que en el método general."""
    if D > 0.80:
        return "Adoptar y, si es útil, compartir / actuar con confianza alta"
    elif 0.50 <= D <= 0.80:
        return "Considerar válido, complementar con más análisis o prueba"
    elif 0.20 <= D < 0.50:
        return "Cuestionar, corregir, contrastar versiones / evidencia"
    else:
        return "Descartar o tratar como no confiable / potencialmente dañino"


def axis_average(df: pd.DataFrame, axis: str) -> float:
    """Promedio de un eje (F, C o P) usando la columna 'valor'."""
    vals = df.loc[df["axis"] == axis, "valor"].astype(float)
    return float(vals.mean())


@dataclass
class TriaxialMatrixResult:
    case_id: str
    title: str
    domain: str
    F: float
    C: float
    P: float
    D: float
    action: str
    table: pd.DataFrame  # criterios completos


In [2]:
def evaluate_with_matrix(case_id: str, title: str, domain: str, 
                         scores: Dict[str, float]) -> TriaxialMatrixResult:
    """
    scores: diccionario { 'F1':0.3, 'F2':0.5, ..., 'P5':0.8 }
    """
    df = criteria_template()
    df["valor"] = df["code"].map(scores)

    if df["valor"].isnull().any():
        missing = df.loc[df["valor"].isnull(), "code"].tolist()
        raise ValueError(f"Faltan valores para: {missing}")

    F_val = axis_average(df, "F")
    C_val = axis_average(df, "C")
    P_val = axis_average(df, "P")

    D_val = (F_val + C_val + P_val) / 3.0
    action = decide_action(D_val)

    return TriaxialMatrixResult(
        case_id=case_id,
        title=title,
        domain=domain,
        F=F_val,
        C=C_val,
        P=P_val,
        D=D_val,
        action=action,
        table=df
    )


In [3]:
# Todos los casos son ficticios.
CASES = [
    {
        "case_id": "L-01",
        "title": "Denuncia formal con evidencia documental y pericial",
        "domain": "legal-penal",
        "summary": (
            "Una persona denuncia acoso laboral y presenta: correos, mensajes, "
            "testigos directos y reporte del área de recursos humanos. "
            "La denuncia fue presentada ante autoridad competente, "
            "y hay expediente abierto con actuaciones formales."
        ),
    },
    {
        "case_id": "L-02",
        "title": "Acusación mediática viral sin denuncia formal",
        "domain": "mediático-redes",
        "summary": (
            "En redes sociales se publica un hilo acusando a un funcionario local de corrupción. "
            "La publicación se hace viral, pero no hay nombre completo, "
            "no se presentan documentos verificables ni existe, hasta el momento, "
            "una denuncia formal registrada ante autoridad."
        ),
    },
    {
        "case_id": "L-03",
        "title": "Conflicto vecinal escalado a denuncia recíproca",
        "domain": "civil-comunitario",
        "summary": (
            "Dos vecinos mantienen un conflicto de años por ruido y uso de áreas comunes. "
            "Ambos presentan denuncias recíprocas por amenazas y daños. "
            "Hay partes policiales previos, pero los testimonios se contradicen "
            "y no existe evidencia clara en video ni audio."
        ),
    },
]

cases_df = pd.DataFrame(CASES)
cases_df


,case_id,title,domain,summary
0,L-01,Denuncia formal con evidencia documental y per...,legal-penal,Una persona denuncia acoso laboral y presenta:...
1,L-02,Acusación mediática viral sin denuncia formal,mediático-redes,En redes sociales se publica un hilo acusando ...
2,L-03,Conflicto vecinal escalado a denuncia recíproca,civil-comunitario,Dos vecinos mantienen un conflicto de años por...


In [4]:
# Diccionario: case_id -> scores {F1..P5}
CASE_SCORES: Dict[str, Dict[str, float]] = {
    # L-01: denuncia formal con evidencia robusta
    "L-01": {
        # FUNDAMENTO
        "F1": 0.9,  # fuente: expediente formal, documentos trazables
        "F2": 0.9,  # consistente con hechos documentados
        "F3": 0.8,  # encaja con dinámicas reales de acoso laboral
        "F4": 0.9,  # no requiere suposiciones fuertes
        "F5": 0.9,  # no contradice evidencia sólida, más bien la acumula

        # CONTEXTO
        "C1": 0.8,  # contexto claro: denuncia, fechas, lugares
        "C2": 0.7,  # hay carga emocional, pero también cauce institucional
        "C3": 0.9,  # no generaliza grupos, se refiere a personas específicas
        "C4": 0.8,  # el caso no nace de campaña mediática, sino de proceso formal
        "C5": 0.8,  # marco realista: trabajo, jerarquías, RRHH

        # PRINCIPIO
        "P1": 0.8,  # menor entropía: seguir proceso formal ordena el conflicto
        "P2": 0.8,  # coherencia interna (hechos) y externa (derecho laboral)
        "P3": 0.9,  # protege dignidad de víctima y debido proceso del denunciado
        "P4": 0.9,  # no busca linchamiento, sino resolución institucional
        "P5": 0.8,  # abre caminos de acción responsable (investigación, medidas)
    },

    # L-02: acusación viral sin denuncia
    "L-02": {
        # FUNDAMENTO
        "F1": 0.3,  # fuente: cuenta de red social, no verificada
        "F2": 0.3,  # no hay consistencia demostrada con hechos conocidos
        "F3": 0.4,  # es plausible que exista corrupción, pero no se prueba aquí
        "F4": 0.2,  # requiere suponer mucho para darle por cierto
        "F5": 0.3,  # podría incluso contradecir datos oficiales, no contrastados

        # CONTEXTO
        "C1": 0.4,  # contexto parcial: hilo viral sin fecha ni documentos claros
        "C2": 0.2,  # alta carga emocional, posible intención de presión
        "C3": 0.7,  # no estigmatiza grupo entero, se dirige a un cargo específico
        "C4": 0.3,  # posible agenda política / mediática detrás
        "C5": 0.5,  # marco realista en abstracto (corrupción existe), pero caso concreto dudoso

        # PRINCIPIO
        "P1": 0.2,  # más entropía: ruido, linchamiento, confusión
        "P2": 0.3,  # poca coherencia entre acusación y pruebas
        "P3": 0.2,  # riesgo fuerte para dignidad y reputación sin debido proceso
        "P4": 0.2,  # alimenta división, desconfianza generalizada
        "P5": 0.3,  # no abre vías claras de acción responsable, más allá de polarizar
    },

    # L-03: conflicto vecinal con denuncias recíprocas
    "L-03": {
        # FUNDAMENTO
        "F1": 0.6,  # fuentes: partes policiales, pero testimonios cruzados
        "F2": 0.5,  # consistencia parcial, muchas contradicciones
        "F3": 0.7,  # la situación es típica y plausible
        "F4": 0.5,  # algunas suposiciones sobre intención de cada parte
        "F5": 0.6,  # no hay evidencia sólida que lo contradiga, pero tampoco lo confirma todo

        # CONTEXTO
        "C1": 0.7,  # contexto claro: vecindad, ruido, áreas comunes
        "C2": 0.4,  # hay mucha emoción, hartazgo y escalamiento
        "C3": 0.9,  # no se ataca a grupos, es conflicto entre individuos
        "C4": 0.7,  # sin agendas políticas, pero sí historia personal
        "C5": 0.8,  # marco realista, muy bien definido

        # PRINCIPIO
        "P1": 0.6,  # se puede reducir entropía con mediación o acuerdos
        "P2": 0.6,  # coherencia si se aplica un esquema de resolución alterna
        "P3": 0.7,  # se puede respetar dignidad de ambas partes
        "P4": 0.8,  # no alimenta odio sistémico; es local y tratable
        "P5": 0.7,  # abre caminos responsables: mediación, regulación de ruido, etc.
    },
}


In [5]:
results: List[TriaxialMatrixResult] = []

for case in CASES:
    cid = case["case_id"]
    scores = CASE_SCORES[cid]
    res = evaluate_with_matrix(
        case_id=cid,
        title=case["title"],
        domain=case["domain"],
        scores=scores,
    )
    results.append(res)

def results_summary(results: List[TriaxialMatrixResult]) -> pd.DataFrame:
    data = []
    for r in results:
        data.append({
            "Case ID": r.case_id,
            "Título": r.title,
            "Dominio": r.domain,
            "F (Fundamento)": round(r.F, 2),
            "C (Contexto)": round(r.C, 2),
            "P (Principio)": round(r.P, 2),
            "D (Discernimiento)": round(r.D, 2),
            "Acción sugerida": r.action,
        })
    return pd.DataFrame(data)

summary_df = results_summary(results)
summary_df


,Case ID,Título,Dominio,F (Fundamento),C (Contexto),P (Principio),D (Discernimiento),Acción sugerida
0,L-01,Denuncia formal con evidencia documental y per...,legal-penal,0.88,0.80,0.84,0.84,"Adoptar y, si es útil, compartir / actuar con ..."
1,L-02,Acusación mediática viral sin denuncia formal,mediático-redes,0.30,0.42,0.24,0.32,"Cuestionar, corregir, contrastar versiones / e..."
2,L-03,Conflicto vecinal escalado a denuncia recíproca,civil-comunitario,0.58,0.70,0.68,0.65,"Considerar válido, complementar con más anális..."


In [6]:
# Buscar el resultado del caso L-02
case_L02 = next(r for r in results if r.case_id == "L-02")
case_L02.table


,axis,code,name,question,valor
0,F,F1,Fuente confiable,La fuente es confiable / verificable,0.3
1,F,F2,Consistencia con hechos,La afirmación es consistente con hechos conocidos,0.3
2,F,F3,Funcionamiento real del mundo,Coincide con el funcionamiento real del mundo,0.4
3,F,F4,Sin suposiciones forzadas,No requiere suposiciones forzadas para sostenerse,0.2
4,F,F5,Sin contradicción fuerte,No contradice evidencia sólida,0.3
5,C,C1,Contexto claro,Existe contexto claro y transparente,0.4
6,C,C2,Sin presión emocional/manipulación,"No proviene de emoción, presión o manipulación",0.2
7,C,C3,Sin estigmatizar grupos,No generaliza ni estigmatiza grupos completos,0.7
8,C,C4,Sin agendas interesadas,Está aislada de agendas políticas o mediáticas...,0.3
9,C,C5,Marco realista,Los datos o hechos están situados en un marco ...,0.5


### Nota didáctica

- Estos casos son **hipotéticos**, pero reflejan patrones reales:
  - denuncias con evidencia formal,
  - acusaciones virales sin sustento,
  - conflictos cotidianos escalados al ámbito legal.

- El objetivo no es “resolver” jurídicamente los casos,
  sino mostrar cómo el **Método Triaxial (F–C–P)**:
  - ayuda a separar ruido de sustancia,
  - hace visible la diferencia entre casos con fundamento y casos solo emocionales,
  - y orienta acciones que disminuyen entropía y aumentan coherencia.

Este notebook puede usarse como base para:
- cursos de ética jurídica,
- análisis de medios y judicialización,
- talleres de pensamiento crítico,
- y desarrollo de herramientas IA para apoyo en análisis de casos.
